In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import zipfile
import requests
import io

In [ ]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip"
response = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(response.content))

In [ ]:
df = pd.read_csv(z.open('SMSSpamCollection'), sep='\t', names=['label', 'message'])

In [ ]:
df['label'] = df['label'].map({'ham': 0, 'spam': 1})
texts = df['message'].values
labels = df['label'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train).toarray()
X_test_tfidf = tfidf_vectorizer.transform(X_test).toarray()

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)
word_index = tokenizer.word_index

In [ ]:
max_len = 100
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_len)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_len)

In [ ]:
w2v_model = Word2Vec(sentences=[text.split() for text in X_train], vector_size=100, window=5, min_count=1, workers=4)
embedding_matrix = np.zeros((len(word_index) + 1, 100))
for word, i in word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]

In [ ]:
def create_model(input_dim, embedding_matrix=None, use_embedding=False):
    model = tf.keras.Sequential()
    if use_embedding:
        model.add(tf.keras.layers.Embedding(input_dim=len(word_index) + 1,
                                            output_dim=100,
                                            weights=[embedding_matrix],
                                            input_length=max_len,
                                            trainable=False))
        model.add(tf.keras.layers.GlobalAveragePooling1D())
    else:
        model.add(tf.keras.layers.Dense(128, input_dim=input_dim, activation='relu'))
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
tfidf_model = create_model(input_dim=X_train_tfidf.shape[1])
tfidf_model.fit(X_train_tfidf, y_train, epochs=10, batch_size=32, validation_data=(X_test_tfidf, y_test), verbose=2)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
140/140 - 5s - 32ms/step - accuracy: 0.9071 - loss: 0.2561 - val_accuracy: 0.9865 - val_loss: 0.0820
Epoch 2/10
140/140 - 3s - 18ms/step - accuracy: 0.9912 - loss: 0.0333 - val_accuracy: 0.9857 - val_loss: 0.0544
Epoch 3/10
140/140 - 4s - 31ms/step - accuracy: 0.9982 - loss: 0.0075 - val_accuracy: 0.9892 - val_loss: 0.0517
Epoch 4/10
140/140 - 2s - 14ms/step - accuracy: 0.9993 - loss: 0.0032 - val_accuracy: 0.9901 - val_loss: 0.0568
Epoch 5/10
140/140 - 3s - 22ms/step - accuracy: 0.9998 - loss: 0.0018 - val_accuracy: 0.9910 - val_loss: 0.0595
Epoch 6/10
140/140 - 3s - 23ms/step - accuracy: 0.9998 - loss: 0.0015 - val_accuracy: 0.9910 - val_loss: 0.0617
Epoch 7/10
140/140 - 2s - 17ms/step - accuracy: 0.9998 - loss: 0.0013 - val_accuracy: 0.9865 - val_loss: 0.0650
Epoch 8/10
140/140 - 2s - 18ms/step - accuracy: 0.9998 - loss: 0.0012 - val_accuracy: 0.9857 - val_loss: 0.0673
Epoch 9/10
140/140 - 2s - 15ms/step - accuracy: 0.9998 - loss: 0.0011 - val_accuracy: 0.9901 - val_loss:

In [ ]:
embedding_model = create_model(input_dim=len(word_index) + 1, embedding_matrix=embedding_matrix, use_embedding=True)
embedding_model.fit(X_train_padded, y_train, epochs=10, batch_size=32, validation_data=(X_test_padded, y_test), verbose=2)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


140/140 - 2s - 12ms/step - accuracy: 0.8658 - loss: 0.5229 - val_accuracy: 0.8664 - val_loss: 0.4566
Epoch 2/10
140/140 - 0s - 3ms/step - accuracy: 0.8658 - loss: 0.4136 - val_accuracy: 0.8664 - val_loss: 0.3774
Epoch 3/10
140/140 - 1s - 5ms/step - accuracy: 0.8658 - loss: 0.3642 - val_accuracy: 0.8664 - val_loss: 0.3495
Epoch 4/10
140/140 - 1s - 5ms/step - accuracy: 0.8658 - loss: 0.3481 - val_accuracy: 0.8664 - val_loss: 0.3432
Epoch 5/10
140/140 - 1s - 5ms/step - accuracy: 0.8658 - loss: 0.3429 - val_accuracy: 0.8664 - val_loss: 0.3379
Epoch 6/10
140/140 - 1s - 9ms/step - accuracy: 0.8658 - loss: 0.3406 - val_accuracy: 0.8664 - val_loss: 0.3352
Epoch 7/10
140/140 - 0s - 3ms/step - accuracy: 0.8658 - loss: 0.3388 - val_accuracy: 0.8664 - val_loss: 0.3336
Epoch 8/10
140/140 - 0s - 3ms/step - accuracy: 0.8658 - loss: 0.3383 - val_accuracy: 0.8664 - val_loss: 0.3327
Epoch 9/10
140/140 - 0s - 2ms/step - accuracy: 0.8658 - loss: 0.3372 - val_accuracy: 0.8664 - val_loss: 0.3325
Epoch 10/10

In [ ]:
tfidf_pred = tfidf_model.predict(X_test_tfidf)
embedding_pred = embedding_model.predict(X_test_padded)

35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [ ]:
tfidf_acc = accuracy_score(y_test, (tfidf_pred > 0.5).astype(int))
embedding_acc = accuracy_score(y_test, (embedding_pred > 0.5).astype(int))

In [ ]:
print(f"TF-IDF Model Accuracy: {tfidf_acc * 100:.2f}%")
print(f"Word2Vec Model Accuracy: {embedding_acc * 100:.2f}%")

TF-IDF Model Accuracy: 99.01%
Word2Vec Model Accuracy: 86.64%
